In [8]:
import tensorflow as tf
import numpy as np
import time
import pandas as pd

# filereader_X = open('feature_vector.csv', 'r')
# feature_matrix = np.loadtxt(filereader_X)
# filereader_X.close()
# 
# filereader_Y = open('expected_output.csv', 'r')
# expected_Y = np.loadtxt(filereader_Y)
# filereader_Y.close()


feature_matrix = pd.read_pickle('feature_vector.pickle')
# expected_Y =  pd.read_pickle('test.pickle')

filereader_Y = open('expected_output.csv', 'r')
expected_Y = np.loadtxt(filereader_Y)
filereader_Y.close()

In [9]:
number_of_features = feature_matrix.shape[1]
number_of_examples = feature_matrix.shape[0]
print("Number of features: {}".format(number_of_features))
print("Number of examples: {}".format(number_of_examples))

Number of features: 1
Number of examples: 18064


In [10]:
# Multilayer Convolutional Network

# You may change these numbers
learning_rate = 0.03
batch_size = 10

n_input = number_of_features
n_hidden_layer_1 = 40
n_hidden_layer_2 = 10
n_output_layer = 1


graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, number_of_features))
    
    # Weights
    weights_layer_1 = tf.Variable(tf.random_normal([n_input, n_hidden_layer_1]))
    weights_layer_2 = tf.Variable(tf.random_normal([n_hidden_layer_1, n_hidden_layer_2]))
    weights_layer_3 = tf.Variable(tf.random_normal([n_hidden_layer_2, n_output_layer]))
    
    # Biases
    bias_layer_1 = tf.Variable(tf.random_normal([n_hidden_layer_1]))
    bias_layer_2 = tf.Variable(tf.random_normal([n_hidden_layer_2]))
    bias_layer_3 = tf.Variable(tf.random_normal([n_output_layer]))
    
    init = tf.initialize_all_variables()
    
    def network_builder(x):
        layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights_layer_1), bias_layer_1))
        layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights_layer_2), bias_layer_2))
        layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights_layer_3), bias_layer_3))
        return layer_3
    
    # Training computation 
    output_layer = network_builder(tf_train_dataset)

    # Define the loss and optimizer, minimize the squared error
    loss = tf.reduce_mean(tf.pow(expected_Y - output_layer, 2))
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

In [6]:
# Run the training process

# You may change this number
num_steps = 151 

start_time = time.time()
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized\n\n")
  
    for step in range(num_steps):
        offset = (step * batch_size) % (number_of_examples - batch_size)
        batch_data = feature_matrix.iloc[range(offset, (offset + batch_size)), range(0, number_of_features)]
        feed_dict = {tf_train_dataset : batch_data}
        _, l = session.run( [optimizer, loss], feed_dict=feed_dict)
        if (step != 0 and step % 25 == 0):
            print( "Minibatch loss at step {}: {}".format(step,l))
        
print("\n Optimization Finished!")
print("--- %s seconds ---" % (time.time() - start_time))

Initialized


Minibatch loss at step 25: 0.046666666865348816
Minibatch loss at step 50: 0.035769857466220856


Minibatch loss at step 75: 0.046666666865348816
Minibatch loss at step 100: 0.046666666865348816

 Optimization Finished!
--- 0.40084004402160645 seconds ---


In [ ]:
# Convert the trained weights and bias into numpy array

with tf.Session() as sess:
    sess.run(init)
    final_weights_layer_1 = weights_layer_1.eval()
    final_weights_layer_2 = weights_layer_2.eval()
    final_weights_layer_3 = weights_layer_3.eval()
    final_bias_layer_1 = bias_layer_1.eval()
    final_bias_layer_2 = bias_layer_2.eval()
    final_bias_layer_3 = bias_layer_3.eval()

In [ ]:
# Create two placeholders (may not be used in this case)

# x = tf.placeholder(tf.float32, shape=[None, 82])
# y_expected = tf.placeholder(tf.float32, shape=[None, 1])

# Initialize weights and biases (may not be used in this case)

# weights = tf.Variable(tf.random_normal([82, 1], stddev=0.35), name="weights")
# biases = tf.Variable(tf.zeros([1]), name="biases")
# # Add an op to initialize the variables.
# init_op = tf.initialize_all_variables()